In [94]:
# -- imports -- #

from jira import JIRA
from jira import JIRAError

import requests
from requests.auth import HTTPBasicAuth
import json

# import ast
# import pandas as pd

In [95]:
# -- authentication functions -- #

def get_auth_info(json_file):
    """takes config.json file, returns auth dictionary of info"""
    
    f= open(json_file)
    data = json.load(f)
    user_info = {}

    for key in data:
        user_info[key] = data[key]

    f.close()
    
    return user_info

def authenticate(user_info):
    """takes user_info dictionary and authenticates through JIRA"""
   
    auth_secret = user_info['password']
    auth_key = user_info['user']
    auth_url = user_info['url']

    auth = HTTPBasicAuth(auth_key, auth_secret)
    
    return auth

def jira_auth(user_info):
    """takes user_info dictionary, returns authentication through JIRA"""
   
    auth_secret = user_info['password']
    auth_key = user_info['user']
    auth_url = user_info['url']
    
    jiraOptions = {'server': auth_url}
    jira = JIRA(options=jiraOptions, basic_auth=(
        auth_key, auth_secret))
    
    return jira

In [96]:
# -- data pull -- #

def get_issue(key):
      item = jira.issue(key, fields='summary,comment,created,issueKey')
      return item

def get_comment_info(comment):
      """takes issue, returns dictionary of information on comment"""

      comment_dict = {
            "body": comment.body,
            "author": { "displayName": comment.author.displayName, "accountId": comment.author.accountId },
            "id": comment.raw['id'],
            "api_call": comment.raw['self'],
            "created": comment.created,
            "updated": comment.updated,
      }
      
      return comment_dict

def get_all_comments(issue):
      
      all_comments = {}
      comments = issue.fields.comment.comments
      
      for comment in comments:
            comment_info = get_comment_info(comment)
            all_comments[comment_info['id']] = comment_info
      
      return all_comments
            

def get_created_date(issue):
      created_date = issue.fields.created

def get_last_one_mo(jira):
      """takes jira, returns array of issue objects from last month""" 
      data_pull = {}
   
      for issue in jira.search_issues('project in ("Web Content & Campaigns") AND createdDate >= startofmonth(-1) AND createdDate <= endOfmonth(-1)', maxResults=100000): # add ", maxResults=1" optional
            data_pull[issue.key] = {}
      return data_pull


# found_issue = get_issue(issue.key)
#       data_pull[issue.key] = {"summary": found_issue.fields.summary,
#                               "comment_info": get_all_comments(found_issue),
#                               "created": get_created_date(found_issue)
#                         }

In [97]:
# -- ACTION: AUTHENTICATE -- #

auth_dict = get_auth_info("config.json")
jira = jira_auth(auth_dict)

In [98]:
# -- ACTION: PULL ISSUES CREATED LAST MONTH -- #

last_month = get_last_one_mo(jira)
# print(len(last_month))

In [99]:
print(last_month)

{'WCC-9433': {}, 'WCC-9411': {}, 'WCC-9332': {}, 'WCC-9299': {}, 'WCC-9298': {}, 'WCC-9297': {}, 'WCC-9296': {}, 'WCC-9295': {}, 'WCC-9294': {}, 'WCC-9292': {}, 'WCC-9291': {}, 'WCC-9290': {}, 'WCC-9289': {}, 'WCC-9288': {}, 'WCC-9287': {}, 'WCC-9286': {}, 'WCC-9285': {}, 'WCC-9284': {}, 'WCC-9283': {}, 'WCC-9282': {}, 'WCC-9281': {}, 'WCC-9279': {}, 'WCC-9278': {}, 'WCC-9277': {}, 'WCC-9276': {}, 'WCC-9275': {}, 'WCC-9274': {}, 'WCC-9273': {}, 'WCC-9272': {}, 'WCC-9271': {}, 'WCC-9270': {}, 'WCC-9269': {}, 'WCC-9268': {}, 'WCC-9267': {}, 'WCC-9266': {}, 'WCC-9265': {}, 'WCC-9264': {}, 'WCC-9262': {}, 'WCC-9261': {}, 'WCC-9260': {}, 'WCC-9259': {}, 'WCC-9258': {}, 'WCC-9257': {}, 'WCC-9256': {}, 'WCC-9255': {}, 'WCC-9254': {}, 'WCC-9253': {}, 'WCC-9252': {}, 'WCC-9251': {}, 'WCC-9250': {}, 'WCC-9249': {}, 'WCC-9247': {}, 'WCC-9246': {}, 'WCC-9245': {}, 'WCC-9244': {}, 'WCC-9243': {}, 'WCC-9242': {}, 'WCC-9241': {}, 'WCC-9240': {}, 'WCC-9239': {}, 'WCC-9238': {}, 'WCC-9237': {}, 'WCC-92

In [100]:
def add_info(hsh_table):
    for key in hsh_table.keys():
        print(key)
        found_issue = jira.issue(key, fields='summary,comment,created,issueKey')
        hsh_table[key] = {"summary": found_issue.fields.summary,
                          "comment_info": get_all_comments(found_issue), 
                          "created": get_created_date(found_issue)}
        print(hsh_table[key])
    
    return hsh_table

In [101]:
last_month_updated = add_info(last_month)

WCC-9433
{'summary': 'Minor updates to Halp pricing page', 'comment_info': {'6414050': {'body': '[~accountid:605af9bf2c761b006901d4e1] - can you please provide more details for this Halp Pricing Page update? What copy needs to be added? Where? Thanks, Natalie', 'author': {'displayName': 'Natalie Oei', 'accountId': '5e96d19745f3660c25e2080a'}, 'id': '6414050', 'api_call': 'https://hello.atlassian.net/rest/api/2/issue/2652520/comment/6414050', 'created': '2022-02-05T08:58:53.703+1100', 'updated': '2022-02-05T08:58:53.703+1100'}, '6486163': {'body': 'hi [~accountid:5e96d19745f3660c25e2080a] following up on our Slack conversation to confirm that we need the following rows removed from the pricing page table:\n\n* “Halp branded Slackbot”\n* “Connect multiple integrations”\n* “Connect multiple Jira Request Types or Zendesk Forms”\n\nThanks\\!', 'author': {'displayName': 'Nicole Pitaro', 'accountId': '605af9bf2c761b006901d4e1'}, 'id': '6486163', 'api_call': 'https://hello.atlassian.net/rest/a

In [102]:
print(last_month_updated)

{'WCC-9433': {'summary': 'Minor updates to Halp pricing page', 'comment_info': {'6414050': {'body': '[~accountid:605af9bf2c761b006901d4e1] - can you please provide more details for this Halp Pricing Page update? What copy needs to be added? Where? Thanks, Natalie', 'author': {'displayName': 'Natalie Oei', 'accountId': '5e96d19745f3660c25e2080a'}, 'id': '6414050', 'api_call': 'https://hello.atlassian.net/rest/api/2/issue/2652520/comment/6414050', 'created': '2022-02-05T08:58:53.703+1100', 'updated': '2022-02-05T08:58:53.703+1100'}, '6486163': {'body': 'hi [~accountid:5e96d19745f3660c25e2080a] following up on our Slack conversation to confirm that we need the following rows removed from the pricing page table:\n\n* “Halp branded Slackbot”\n* “Connect multiple integrations”\n* “Connect multiple Jira Request Types or Zendesk Forms”\n\nThanks\\!', 'author': {'displayName': 'Nicole Pitaro', 'accountId': '605af9bf2c761b006901d4e1'}, 'id': '6486163', 'api_call': 'https://hello.atlassian.net/re

In [103]:
d = open("last_month_updated.json", "w")
json.dump(last_month_updated, d)
d.close()